In [16]:
suppressPackageStartupMessages({
    
    library(dplyr)
    library(tidyr)
    library(UpSetR)
    library(ggplot2)
    library(ComplexHeatmap)
    library(reshape2)
    library(corrr)
    library(assertthat)

    
})

gtex_chr21=readRDS(here::here('output/gtex/plier_result_k1_frac0.7_chr21.rds'))

In [17]:
nrow(gtex_chr21$Z)

[1] 17548

In [18]:
head(gtex_chr21$summary) 

,pathway,LV index,AUC,p-value,FDR
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,REACTOME_GENERIC_TRANSCRIPTION_PATHWAY,1,0.7765794,5.336776e-15,9.803553e-14
2,REACTOME_DNA_REPAIR,1,0.6662390,3.840336e-03,7.660792e-03
3,REACTOME_METABOLISM_OF_RNA,1,0.7033541,1.722206e-07,7.905179e-07
4,REACTOME_TRANSCRIPTION,1,0.6231761,5.607844e-03,1.076859e-02
5,MIPS_SPLICEOSOME,1,0.8119698,3.120247e-09,2.004050e-08
6,KEGG_SPLICEOSOME,1,0.6780828,8.807599e-04,2.010988e-03


In [19]:
head(gtex_chr21$Z)

GAS6,0.053533337,0.00000000,0.012807752,0.00000000,0.00000000,0.00000000,0.00000000,0.008485100,0.00000000,0.000000000,⋯,0.0000000,0.000000000,0.024873684,0.06458124,0.03462393,0.00000000,0.0000000,0.38756952,0.13953691,0.00000000
MMP14,0.156937401,0.00000000,0.000000000,0.01384223,0.00000000,0.12232310,0.00000000,0.152617574,0.01147268,0.000000000,⋯,0.0000000,0.000000000,0.000000000,0.00000000,0.00000000,0.01644112,0.0000000,0.53960829,0.00000000,0.00000000
MARCKSL1,0.000000000,0.04185200,0.000000000,0.00000000,0.00960182,0.00000000,0.13434774,0.000000000,0.00000000,0.004402272,⋯,0.0000000,0.005064292,0.014862108,0.01001471,0.01240426,0.00000000,0.0000000,0.00000000,0.00000000,0.04098988
SPARC,0.002933146,0.00000000,0.001542308,0.00000000,0.01341513,0.06677496,0.00000000,0.034412807,0.00000000,0.000000000,⋯,0.0000000,0.000000000,0.000000000,0.06377960,0.00000000,0.00000000,0.0000000,0.07948821,0.00000000,0.00000000
CTSD,0.000000000,0.00000000,0.019784559,0.00000000,0.18671529,0.00778830,0.00000000,0.000000000,0.00000000,0.000000000,⋯,0.1564862,0.000000000,0.002756522,0.03089390,0.13391500,0.00000000,0.1120276,0.00000000,0.06755147,0.00000000
EPAS1,0.000000000,0.03983347,0.000000000,0.00000000,0.00000000,0.03585003,0.05580654,0.005876182,0.04367746,0.000000000,⋯,0.0000000,0.000000000,0.000000000,0.00000000,0.00000000,0.00000000,0.0000000,0.00000000,0.00000000,0.00000000


In [20]:
# Function to get top N ranked genes for each LV as a data frame
getTopRankedGenesDataFrame <- function(matrix, n = 10) {
  # Sorting and getting top n indices for each column
  topRankedIndices <- apply(matrix, 2, function(column) order(column, decreasing = TRUE)[1:n])
  
  # Creating an empty data frame to store top ranked gene names
  topRankedGenesDF <- as.data.frame(matrix(nrow = n, ncol = ncol(matrix)))
  
  # Filling the data frame with gene names
  for (i in 1:ncol(matrix)) {
    topRankedGenesDF[, i] <- rownames(matrix)[topRankedIndices[, i]]
  }
  
  # Naming the columns as LV1, LV2, ..., LVk
  colnames(topRankedGenesDF) <- paste0("LV", 1:ncol(matrix))
  
  return(topRankedGenesDF)
}

# Applying the function to your gtex_chr21$Z matrix
topRankedGenesDF <- getTopRankedGenesDataFrame(gtex_chr21$Z, 10)

# Print the data frame to see the results
head(topRankedGenesDF)


,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV1139,LV1140,LV1141,LV1142,LV1143,LV1144,LV1145,LV1146,LV1147,LV1148
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,ALKBH1,CYP19A1,C12orf71,TRPC6,HIST1H4D,KIF18B,DIO1,UGT3A2,LCN8,RETNLB,⋯,CFLAR-AS1,STATH,SUMO4,OR2K2,ZNF442,KRT76,MYLK2,MME,MAGEA5,GUCY1B2
2,ZNF526,SPATA20P1,TBC1D29,LHCGR,CD177,RRM2,BMP8A,KRT77,SSPO,SPINK4,⋯,TAS2R39,C6orf58,FAS-AS1,IL9,ZNF844,KRT3,SMTNL1,TWIST2,HOXD12,SLC44A5
3,ZNF564,LINC01426,LINC00479,SLITRK3,KCNE1,IQGAP3,TECTB,IL37,CHAD,CLCA1,⋯,USF1P1,GPRC6A,BLID,SPTBN5,PPDPF,CLDN17,CHRNA10,OPN3,SALL3,RPL3P1
4,ZNF227,IGF1,OR10D3,RAB40A,LPAR2,KIF4A,DUOX2,LGALSL,PRCD,MUC2,⋯,AIRN,ASCL3,HYMAI,AATK,C7orf50,PITX2,CLTCL1,WIPI1,CPA6,TEKT4P2
5,ZNF443,CCL13,LINC00308,ADRB3,SMPD5,CDK1,S100A5,GJB4,DGCR10,B3GNT6,⋯,GCNT7,BPIFB2,SNORA28,SYT10,ZNF627,FMO9P,OPTN,CTHRC1,ISL1,OR10A3
6,LYSMD1,POU5F1P4,TMEM89,EPHA7,HK3,SPC25,MT1F,ABHD12B,PCSK9,FCGBP,⋯,NCR1,SMR3B,SNURF,SERPINB9,XPO1,SERPINB11,ART1,DUSP10,PI16,PKD1L1


In [21]:
head(gtex_chr21$B)

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
"1,REACTOME_GENERIC_TRANSCRIPTION_PATHWAY",0.187422456,-0.308640457,-0.04479051,0.37611445,-0.074696554,-0.03383109,0.6697872978,0.493554295,0.328171971,0.18752966,⋯,-0.221648965,-0.120663485,0.115484780,-0.1130431165,-0.05870664,0.55017119,-0.059750264,0.112675394,-0.16071672,0.12276441
LV 2,-0.046944533,-0.075768201,-0.20699344,0.17505346,0.007132952,-0.03968156,-0.0481128654,0.031141556,-0.053178864,0.01789782,⋯,-0.007212119,-0.008661326,-0.014651064,0.0247549779,0.05458687,-0.02076561,-0.018060914,-0.075166837,-0.02938883,-0.12538513
"3,chr21",-0.107883825,-0.028366814,-0.07064137,-0.01272791,-0.004784065,-0.00445917,-0.0145780910,-0.041467583,-0.039117826,-0.03067244,⋯,-0.003652945,0.011791271,-0.030559158,-0.0003855234,0.01934153,-0.05982883,-0.003022672,-0.004455461,-0.01377039,-0.02566878
LV 4,0.079776160,-0.002247112,0.02878914,-0.05344902,0.001574073,-0.01609270,-0.0895926780,-0.007017755,0.002492495,0.03312515,⋯,-0.015155624,-0.029412875,0.008317372,0.0042721063,0.65611869,0.08280698,0.002217951,-0.085985168,0.01431606,-0.02011327
"5,IRIS_Neutrophil-Resting",-0.073098503,0.002056207,-0.02949975,-0.03962876,0.009217471,-0.08726276,0.0001216234,-0.016896404,-0.042021175,-0.05800727,⋯,0.009203931,-0.027090183,-0.032153759,-0.0019166909,0.02289051,0.08718488,-0.016767408,0.011110693,0.03204665,-0.09214235
"6,GSE15750_DAY6_VS_DAY10_TRAF6KO_EFF_CD8_TCELL_UP",-0.006940604,0.036053115,0.09730621,-0.09347276,0.039415997,-0.05719299,0.0050925657,0.114702579,-0.066412911,-0.04330553,⋯,-0.028510147,-0.178248470,-0.039265746,-0.0267294378,-0.07048049,-0.06548119,0.236635483,0.016231646,-0.07462165,-0.03442109


In [22]:
m_summary=gtex_chr21$summary

In [23]:
int_LV=m_summary[grep('chr21', m_summary$pathway),] %>%
dplyr::arrange(FDR) %>%
dplyr::filter(FDR < 0.05 & AUC > 0.7) %>%
dplyr::select(`LV index`) %>%
unique()

int_LV=as.character(as.list(int_LV))

In [24]:
int_LV

[1] "c(\"146\", \"193\", \"148\", \"806\", \"799\", \"988\", \"400\", \"947\", \"910\", \"300\")"

In [25]:
library(dplyr)
library(stringr)

int_LV <- c("146", "193", "148", "806", "799", "988", "400", "947", "910", "300")

m_summary %>%
  mutate(`LV index` = as.character(`LV index`)) %>%
  filter(str_trim(`LV index`) %in% int_LV)

pathway,LV index,AUC,p-value,FDR
<chr>,<chr>,<dbl>,<dbl>,<dbl>
chr21_q22.3,146,0.8938836,1.603403e-16,3.861832e-15
chr21,146,0.5624634,5.513756e-03,1.060954e-02
chr21_q22.11,148,0.8129491,1.365591e-08,7.702915e-08
chr21,148,0.6664609,9.488786e-13,1.164196e-11
GSE18148_CBFB_KO_VS_WT_TREG_UP,193,0.5066561,4.390604e-01,5.080325e-01
GSE7460_CTRL_VS_FOXP3_OVEREXPR_TCONV_1_UP,193,0.4562906,8.464046e-01,8.753133e-01
chr21_q22.11,193,0.8499625,3.647248e-10,2.736405e-09
chr21,193,0.6097534,2.081402e-06,7.804568e-06
chr21_q22.2,300,0.7197070,7.684038e-03,1.427060e-02


In [26]:
m_summary %>%
  mutate(`LV index` = as.character(`LV index`)) %>%
  filter(str_trim(`LV index`) %in% int_LV) %>%
  dplyr::filter(`LV index` == '193')

pathway,LV index,AUC,p-value,FDR
<chr>,<chr>,<dbl>,<dbl>,<dbl>
GSE18148_CBFB_KO_VS_WT_TREG_UP,193,0.5066561,4.390604e-01,5.080325e-01
GSE7460_CTRL_VS_FOXP3_OVEREXPR_TCONV_1_UP,193,0.4562906,8.464046e-01,8.753133e-01
chr21_q22.11,193,0.8499625,3.647248e-10,2.736405e-09
chr21,193,0.6097534,2.081402e-06,7.804568e-06


In [28]:
gtex_chr21$Z %>%
data.frame() %>%
dplyr::select(X193) %>%
dplyr::arrange(desc(X193)) %>%
head(20)

,X193
,<dbl>
KRTAP19-5,5.596972
KRTAP19-7,4.985712
KRTAP7-1,4.945952
KRTAP19-3,4.842241
KRTAP8-1,4.769247
KRTAP19-1,3.985043
KRT38,3.926521
KRT37,3.414393
KRTAP19-4,3.131042


In [25]:
head(gtex_chr21$B[grep('chr21', rownames(gtex_chr21$B)),])

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
"3,chr21",-0.107883825,-0.0283668139,-0.0706413709,-0.012727912,-0.004784065,-0.004459170,-0.014578091,-0.041467583,-0.039117826,-0.030672439,⋯,-0.003652945,0.011791271,-0.030559158,-0.0003855234,0.019341534,-0.05982883,-0.003022672,-0.004455461,-0.013770389,-0.02566878
"31,chr21_q22.3",-0.015945946,-0.0261468987,-0.0008196497,-0.007616873,-0.009104582,-0.006871012,0.003830322,-0.015632837,-0.003193543,-0.022296321,⋯,-0.002484197,0.003069885,0.002942400,0.0001768184,0.000433234,-0.01168156,-0.013129472,-0.001846661,-0.007411058,-0.01097021
"42,chr21_q21.3",-0.063052216,-0.0319299666,-0.0543655724,-0.017606158,0.009644670,-0.057147326,0.044763799,-0.065220545,0.030105057,-0.003920223,⋯,-0.005532234,-0.024486883,0.018279149,-0.0027642230,0.025228742,-0.04608153,0.030894541,-0.041149477,0.013933672,0.02526394
"82,chr21",-0.008932274,0.0004740615,0.0411873290,0.009368512,-0.008202127,-0.035316699,-0.070304229,0.008714766,-0.019940752,-0.006476070,⋯,0.003459387,0.005515774,-0.012119112,0.0117059734,0.017108954,0.03201239,0.016873884,0.019228671,0.014293764,0.04254612
"109,chr21",0.016018093,0.0259009953,0.0231994031,0.005590103,-0.002988433,0.015531055,-0.146388897,-0.069033947,-0.003236288,-0.011657327,⋯,0.002862471,-0.040969496,-0.039472752,-0.0116448833,-0.010572670,-0.02910418,-0.012094230,-0.015486515,0.002453114,0.00948096
"118,chr21_q22.11",0.026091637,0.0108230734,0.0214149482,-0.003391445,0.007025638,-0.013906160,-0.001996191,-0.008860263,0.004480764,-0.038806897,⋯,0.006787230,-0.002876459,-0.009168482,-0.0010667046,-0.005203027,0.01773322,-0.001590433,-0.001290199,0.013175252,0.01728443


In [27]:
colnames(gtex_chr21)[colnames(gtex_chr21) %in% paste0('LV',c(3,31,42,82,109,118))]colnames(gtex_chr21)

NULL

In [31]:
c(3,31,42,82,109,118)]

GAS6,0.012807752,0.000000000,0.029702255,0.010882386,0.0000000000,0.000000000
MMP14,0.000000000,0.015497392,0.010251441,0.000000000,0.0166884665,0.000000000
MARCKSL1,0.000000000,0.013656139,0.027730859,0.000000000,0.0398948183,0.000000000
SPARC,0.001542308,0.007382264,0.000000000,0.000000000,0.0000000000,0.000000000
CTSD,0.019784559,0.004421192,0.000000000,0.000000000,0.0000000000,0.000000000
EPAS1,0.000000000,0.000000000,0.000000000,0.000000000,0.0200965082,0.002835021
PALLD,0.028356031,0.000000000,0.000000000,0.001197321,0.0000000000,0.000000000
PHC2,0.071887401,0.000000000,0.000000000,0.000000000,0.0000000000,0.000000000
LGALS3BP,0.005627914,0.000000000,0.000000000,0.000000000,0.0000000000,0.000000000
SERPING1,0.000000000,0.000000000,0.000000000,0.000000000,0.0147714485,0.005473834
TGM2,0.023242910,0.003609186,0.000000000,0.055655563,0.0289400361,0.000000000


In [45]:
library(dplyr)

# Define the specific columns you're interested in
columns_of_interest <- c(3, 31, 42, 82, 109, 118)

# Initialize an empty list to store the dataframes
list_of_dfs <- list()

for (i in columns_of_interest) {
  # Create a dataframe for each specified column
  tmp_df <- data.frame(Gene = rownames(gtex_chr21$Z), LV = gtex_chr21$Z[,i]) %>%
    arrange(desc(LV))
  
  # Reset row names to be sequential from 1 to nrow(tmp_df)
  rownames(tmp_df) <- seq_len(nrow(tmp_df))
  
  # Rename the 'LV' column to reflect the column index (e.g., LV3, LV31, etc.)
  colnames(tmp_df)[2] <- paste0("LV", i)
  
  # Store the dataframe in the list
  list_of_dfs[[paste0("LV", i)]] <- tmp_df
}

# To access a specific dataframe, use its name in the list, for example:
lapply(list_of_dfs, head) # Display the first few rows of the dataframe for LV3


,Gene,LV3
,<chr>,<dbl>
1,C12orf71,1.883230
2,TBC1D29,1.789591
3,LINC00479,1.680634
4,OR10D3,1.664092
5,LINC00308,1.655354
6,TMEM89,1.627016
,Gene,LV31
,<chr>,<dbl>
1,KRTAP24-1,4.144923
